In [1]:
# !hostname -I

In [2]:
# !pip install selenium
# !pip install hdfs

In [3]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

from bs4 import BeautifulSoup as bs
from hdfs import InsecureClient
from datetime import datetime
import pandas as pd

# Chrome 옵션 설정
options = Options()
# options.binary_location = '/usr/local/bin/chrome-headless-shell' # 크롬브라우저경로
options.binary_location = r'C:\Users\plctoit\chrome\chrome.exe ' # 크롬브라우저경로
# options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage") # 공유 메모리 비활성화
options.add_argument("--disable-gpu") # GPU 비활성화
options.add_argument("--window-size=1920,1080") # 가상 화면 크기설정

# ChromeDriver 설정
service = Service(r'C:\Users\plctoit\chrome\chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

driver.get('https://www.starbucks.co.kr/index.do')
print(driver.title)

Starbucks Korea


In [4]:
hover_element = driver.find_element(By.CSS_SELECTOR,"#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a")
menu_option = driver.find_element(By.CSS_SELECTOR,"#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a")

# 마우스 호버 액션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

# 약간의 대기 시간 (메뉴가 나타날 시간 필요)
time.sleep(1)
print("매장검색중...")

menu_option.click()
print('페이지 이동')
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(6) > a').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, '#mCSB_2_container > ul > li:nth-child(1) > a').click()
print('부산지역검색중')
time.sleep(3)


매장검색중...
페이지 이동
부산지역검색중


In [6]:
# 현재 페이지 정보가져오기
star_list=[]
html = driver.page_source
soup = bs(html, 'html.parser')
contents = soup.select('#mCSB_3_container > ul > li')
for c in contents:
    star_list.append([c['data-name'], c['data-lat'], c['data-long'],c.select_one('p.result_details').text,c.select_one('i')['class'][0]])
driver.quit()

In [18]:
columns = ['name','lat','lon','addr','stype']
starbucks = pd.DataFrame(star_list, columns=columns)
starbucks.head(25)

,name,lat,lon,addr,stype
0,금정남산동DT,35.2621786292157,129.091325043351,부산광역시 금정구 중앙대로 1989 (남산동)1522-3232,pin_generalDT
1,금정구청DT,35.2388135211636,129.092985849499,부산광역시 금정구 중앙대로 1730 (부곡동)1522-3232,pin_generalDT
2,온천장역DT,35.224006,129.08656,부산광역시 금정구 식물원로 13 (장전동)1522-3232,pin_generalDT
3,부산대정문,35.23212165114605,129.0844959292725,부산광역시 금정구 부산대학로64번길 12 (장전동)1522-3232,pin_general
4,부산대역,35.23052180284867,129.08869433543111,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)1522-3232",pin_general
5,금정이마트,35.249819818853915,129.09078227309334,부산광역시 금정구 중앙대로1841번길 24 (구서동)1522-3232,pin_general
6,금정구서,35.246908383445785,129.08873844189125,부산광역시 금정구 금강로 441 (구서동)1522-3232,pin_general
7,부산서동,35.21463924851766,129.10558442984572,부산광역시 금정구 서동로 173 (서동)1522-3232,pin_general
8,용호동W스퀘어,35.1337011,129.113324,"부산광역시 남구 분포로 145 (용호동) 더블유스퀘어동 1019호,1020호,102...",pin_general
9,동명대DT,35.12311959047579,129.09901642703608,부산광역시 남구 신선로 423 (용당동)1522-3232,pin_generalDT


In [17]:
starbucks.info()
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
csv_path = f"starbucks_{timestamp}.csv"
starbucks.to_csv(csv_path,index=True)
print('csv 변환완료')
hdfs = InsecureClient('http://namenode:9870',user='hadoop')
path = f"/user/hadoop/{csv_path}"
try:
    hdfs.upload(path,csv_path)
    print('하둡 업로드 완료!')
except:
    print('업로드 실패!')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    143 non-null    object
 1   lat     143 non-null    object
 2   lon     143 non-null    object
 3   addr    143 non-null    object
 4   stype   143 non-null    object
dtypes: object(5)
memory usage: 5.7+ KB
csv 변환완료
하둡 업로드 완료!
